In [32]:
import sys

origin = 'data/Aldhani/eoagritwin/'
sys.path.append('/home/potzschf/repos/')

from helperToolz.polygons_to_labels import *
from helperToolz.helpsters import *
from helperToolz.dicts_and_lists import * 

gtiff_driver = gdal.GetDriverByName('GTiff')

In [ ]:
# get the Brandenburg 2019 comps for test run (take away the IACS data)
state = 'BRB'
year = 2019
outFolder = path_safe(f'/data/Aldhani/eoagritwin/fields/output/trash/for_Shreya/{state}/{year}/')
force_tifs = getFilelist(f'/data/Aldhani/eoagritwin/force/output/{state}/{year}/X0067_Y0041/FBM/', '.tif')

ds= gdal.Open(force_tifs[0])
XX = ds.RasterXSize
YY = ds.RasterYSize
gt = ds.GetGeoTransform()
prj = ds.GetProjection()

colorList = ['BLU', 'GRN', 'RED', 'BNR']

In [35]:
for month in [3,6,8]:# range(3,9,1): # nur march, june, august
    monthly = [tif for tif in force_tifs if f'-{month:02d}.tif' in tif]
    MM = (INT_TO_MONTH[f'{month:02d}'])
    # export labels as tif chips
    out_ds = gtiff_driver.Create((f'{outFolder}{state}_{year}_{MM}.tif'), XX, YY, 4, gdal.GDT_Float32)

    out_ds.SetGeoTransform(gt)
    out_ds.SetProjection(prj)


    for band, color in enumerate(colorList):
        for file in monthly:
            if color == file.split('_SEN2H_')[-1].split('_FBM')[0]:
                dum_ds = gdal.Open(file)
                arr = dum_ds.GetRasterBand(1).ReadAsArray()
                out_ds.GetRasterBand(band + 1).WriteArray(arr)
                out_ds.GetRasterBand(band + 1).SetDescription(color)

    del out_ds

In [37]:
tfiles = [tfile for tfile in getFilelist(outFolder, '.tif') if all(substr in tfile for substr in [str(year), state])]

In [40]:
tfiles[0].split('_')[-1].split('.')[0]

'August'

In [ ]:
# cut them in chipsize * chipsize chips
chip_size = 256

tfiles = [tfile for tfile in getFilelist(outFolder, '.tif') if all(substr in tfile for substr in [str(year), state])]
for tfile in tfiles:
    ds = gdal.Open(tfile)
    arr = stackReader(tfile)
    row_col_ind =get_row_col_indices(chip_size, 0, arr.shape[0], arr.shape[1])
    row_start = row_col_ind[0]
    row_end   = row_col_ind[1]
    col_start = row_col_ind[2]
    col_end   = row_col_ind[3]

    geoTF = ds.GetGeoTransform()
    prj = ds.GetProjection()

    mm = tfile.split('_')[-1].split('.')[0]
    chip_path = path_safe(f'{outFolder}{mm}/')
    
    
    for i in range(len(row_end)):
        for j in range(len(col_end)):

            # export as tif chips
            out_ds = gtiff_driver.Create(f'{chip_path}{state}_{year}_{mm}_rs{row_start[i]:04d}_cs{col_start[j]:04d}.tif',
                                         int(chip_size), int(chip_size), arr.shape[-1], gdal.GDT_Float32)
            # change the Geotransform for each chip
            geotf = list(geoTF)
            # get column and rows from filenames
            geotf[0] = geotf[0] + geotf[1] * col_start[j]
            geotf[3] = geotf[3] + geotf[5] * row_start[i]
            
            out_ds.SetGeoTransform(tuple(geotf))
            out_ds.SetProjection(prj)
            for b in range(arr.shape[-1]):
                out_ds.GetRasterBand(b+1).WriteArray(arr[row_start[i]:row_end[i], col_start[j]:col_end[j], b])
                out_ds.GetRasterBand(b+1).SetDescription(colorList[b]) 
            del out_ds
    

In [ ]:
prefix = '/data/Aldhani/eoagritwin/'
state = 'SL'
year = 2021

f"{prefix}/fields/Auxiliary/vrt/{state}/{year}/{dirfinder(f'{prefix}/fields/Auxiliary/vrt/{state}/{year}/')[0]}"

In [ ]:
aa = []
for file in getFilelist('/data/Aldhani/eoagritwin/fields/ai4boundaries/sentinel2/masks/', '.tif', deep=True):
    ds = gdal.Open(file)
    arr = ds.GetRasterBand(1).ReadAsArray()

    if np.nansum(arr) == 0:
        aa.append(1)